# 🤖 Homework 10: Business Agent
**MIS 769 - Big Data Analytics for Business | Spring 2026**

**Points:** 20 | **Due:** Sunday, April 19, 2026 @ 11pm Pacific

**Author:** Richard Young, Ph.D. | UNLV Lee Business School

**Compute:** CPU (free tier)

---

## What You'll Learn

1. What AI agents are and how they work
2. Build an agent that can use tools to accomplish tasks
3. Design agents for specific business workflows
4. Implement the ReAct (Reasoning + Acting) pattern

---

## Part 1: Agent Setup (3 points)

In [ ]:
!pip install transformers torch -q

from transformers import pipeline
import json

# Load a reasoning model
llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=150
)

print("✅ Language model loaded")

In [ ]:
# Define a simple simulated business environment
PRODUCTS = {
    "LP001": {"name": "TechPro Laptop", "price": 899, "stock": 15, "category": "Electronics"},
    "LP002": {"name": "UltraBook Pro", "price": 649, "stock": 23, "category": "Electronics"},
    "LP003": {"name": "GamerMax", "price": 1299, "stock": 8, "category": "Electronics"},
    "PH001": {"name": "SmartPhone X", "price": 799, "stock": 45, "category": "Electronics"},
    "AC001": {"name": "Wireless Earbuds", "price": 129, "stock": 100, "category": "Accessories"},
}

ORDERS = {
    "ORD001": {"product": "LP001", "status": "shipped", "customer": "John"},
    "ORD002": {"product": "PH001", "status": "processing", "customer": "Jane"},
    "ORD003": {"product": "AC001", "status": "delivered", "customer": "Bob"},
}

print("🏪 BUSINESS ENVIRONMENT")
print("=" * 60)
print(f"Products: {len(PRODUCTS)}")
print(f"Orders: {len(ORDERS)}")

## Part 2: Tool Design (5 points)

In [ ]:
# Define tools the agent can use
def search_products(query):
    """Search for products by name or category."""
    query = query.lower()
    results = []
    for pid, product in PRODUCTS.items():
        if query in product['name'].lower() or query in product['category'].lower():
            results.append(pid)
    return results if results else "No products found"

def get_price(product_id):
    """Get the price of a product."""
    if product_id in PRODUCTS:
        return f"${PRODUCTS[product_id]['price']}"
    return "Product not found"

def check_stock(product_id):
    """Check inventory for a product."""
    if product_id in PRODUCTS:
        stock = PRODUCTS[product_id]['stock']
        return f"{stock} units in stock"
    return "Product not found"

def get_order_status(order_id):
    """Get the status of an order."""
    if order_id in ORDERS:
        order = ORDERS[order_id]
        return f"Order {order_id}: {order['status']} (Product: {order['product']})"
    return "Order not found"

def list_all_products():
    """List all available products."""
    return list(PRODUCTS.keys())

# Tool registry
TOOLS = {
    "search_products": search_products,
    "get_price": get_price,
    "check_stock": check_stock,
    "get_order_status": get_order_status,
    "list_all_products": list_all_products
}

print("🔧 AVAILABLE TOOLS")
print("=" * 60)
for name, func in TOOLS.items():
    print(f"  • {name}: {func.__doc__}")

In [ ]:
# Test the tools
print("🧪 TOOL TESTS")
print("=" * 60)

print(f"\nsearch_products('laptop'): {search_products('laptop')}")
print(f"get_price('LP002'): {get_price('LP002')}")
print(f"check_stock('LP002'): {check_stock('LP002')}")
print(f"get_order_status('ORD001'): {get_order_status('ORD001')}")

## Part 3: ReAct Loop (5 points)

In [ ]:
def parse_action(response):
    """Parse the agent's response to extract action and argument."""
    response = response.lower().strip()
    
    # Try to find tool calls
    for tool_name in TOOLS.keys():
        if tool_name in response:
            # Try to extract argument
            if '(' in response and ')' in response:
                start = response.find('(') + 1
                end = response.find(')')
                arg = response[start:end].strip().strip('"').strip("'")
                return tool_name, arg
            return tool_name, ""
    
    return None, None

def execute_tool(tool_name, arg):
    """Execute a tool and return the result."""
    if tool_name not in TOOLS:
        return f"Unknown tool: {tool_name}"
    
    try:
        if arg:
            return TOOLS[tool_name](arg)
        else:
            return TOOLS[tool_name]()
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Action parser ready")

In [ ]:
def run_agent(task, max_steps=5):
    """Run the agent on a task using ReAct pattern."""
    print(f"📋 TASK: {task}")
    print("=" * 60)
    
    tool_descriptions = "\n".join([f"- {name}: {func.__doc__}" for name, func in TOOLS.items()])
    
    history = []
    
    for step in range(max_steps):
        # Build prompt
        prompt = f"""You are a helpful business assistant with these tools:
{tool_descriptions}

Task: {task}

Previous steps:
{chr(10).join(history) if history else 'None'}

What should you do next? If you have enough information, say FINAL ANSWER: [your answer].
Otherwise, use a tool by saying: USE tool_name(argument)

Response:"""
        
        # Get LLM response
        response = llm(prompt)[0]['generated_text']
        print(f"\n🤔 THOUGHT {step+1}: {response}")
        
        # Check for final answer
        if 'final answer' in response.lower():
            print(f"\n✅ FINAL ANSWER: {response}")
            return response
        
        # Parse and execute action
        tool_name, arg = parse_action(response)
        
        if tool_name:
            print(f"🔨 ACTION: {tool_name}({arg})")
            result = execute_tool(tool_name, arg)
            print(f"👁️ OBSERVATION: {result}")
            history.append(f"Step {step+1}: Used {tool_name}({arg}) -> {result}")
        else:
            # If no tool found, treat as reasoning step
            history.append(f"Step {step+1}: Thought - {response}")
    
    print("\n⚠️ Max steps reached")
    return "Could not complete task"

print("✅ Agent ready")

In [ ]:
# Test the agent
run_agent("Find the cheapest laptop in stock")

## Part 4: Business Task (5 points)

In [ ]:
# Run agent on business scenarios
print("\n" + "="*60)
run_agent("What is the status of order ORD001?")

In [ ]:
print("\n" + "="*60)
run_agent("How many electronics products do we have?")

In [ ]:
# More complex query
print("\n" + "="*60)
run_agent("Check if the GamerMax laptop is in stock and what it costs")

## Part 5: Error Handling (2 points)

In [ ]:
# Test error handling
print("📊 ERROR HANDLING TESTS")
print("=" * 60)

# Invalid product
print(f"\nInvalid product: {get_price('INVALID')}")

# Invalid order
print(f"Invalid order: {get_order_status('INVALID')}")

# Empty search
print(f"Empty search: {search_products('xyz123')}")

In [ ]:
# Agent with difficult query
print("\n" + "="*60)
run_agent("Find a product called 'SuperComputer'")

---

## Questions to Answer

**Q1:** Describe your agent's tools. Why did you choose them?

*Your answer:*

**Q2:** Walk through one complete agent trace. What worked well?

*Your answer:*

**Q3:** When did the agent fail? How would you improve it?

*Your answer:*

**Q4:** How could this agent save time in a real business?

*Your answer:*

---

## Submission Checklist

| Item | Points | Done? |
|------|--------|-------|
| Part 1: Agent setup | 3 | ☐ |
| Part 2: Tool design | 5 | ☐ |
| Part 3: ReAct loop | 5 | ☐ |
| Part 4: Business task | 5 | ☐ |
| Part 5: Error handling | 2 | ☐ |
| **Total** | **20** | |